In [32]:
import pandas as pd
import re
import langid
import string
import math
DOC_FILE = '/home/antonio/lattes/artigos_22072019.csv'
TOPIC_FILE = '../w2v/resultados/all_titles_sw_titles/results/titles/result_topic_10.txt'

## Read Documents

In [2]:
docs = pd.read_csv(DOC_FILE,sep='|', encoding='utf-8', engine='python')

In [3]:
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

In [4]:
def to_lower(text):
    text = [word.lower() for word in text.split()]
    return " ".join(text)

In [5]:
def remove_invalid(text):
    
    noise_file = open('../noise.txt', 'r')
    noise = noise_file.read()
    
    text = remove_punctuation(text)
    text = to_lower(text)
    text = re.sub('\n', ' ', text)
    text = re.sub(' +', ' ', text)
    
    text = text.split(' ')
    
    text2 = ''
    
    for index, word in enumerate(text):
        if(len(word) < 2 or len(word) > 16):
            text.pop(index)
        elif word in noise:
            text.pop(index)
        elif re.search('\d+', word):
            text.pop(index)
    for word in text:
        text2 += word + ' '
    text2 = text2[:-1]
    
    return text2

In [6]:
natural_docs = []
for row in docs.iterrows():
    if row[1][4] and row[1][5] == 'Inglês':  
        natural_docs.append(row[1][4])
docs = None

In [7]:
english_docs = []
for document in natural_docs:
    lang, log_prob = langid.classify(document)
    if lang == 'en':
        english_docs.append(document)
natural_docs = None

In [8]:
df = pd.DataFrame(columns = ['Titles', 'Clean', 'Topic'])
df['Titles'] = english_docs
df

,Titles,Clean,Topic
0,Isolating Complex Polynomial Roots: An algebra...,NaN,NaN
1,An algebraic algorithm to Isolate Complex poly...,NaN,NaN
2,Using Computer Algebra and Chebyshev Polynomia...,NaN,NaN
3,Reviewing the SacarWeb design based in discoun...,NaN,NaN
4,Extended Immersive Learning Environment: A Hyb...,NaN,NaN
...,...,...,...
57973,A multicriteria approach for analysis of confl...,NaN,NaN
57974,A Multicriteria Approach Using MAUT to Assist ...,NaN,NaN
57975,A Voting Approach Applied to Preventive Mainte...,NaN,NaN
57976,Preference modeling experiments with surrogate...,NaN,NaN


In [9]:
for index, row in df.iterrows():
    df.loc[index,'Clean'] = remove_invalid(row[0])
df

,Titles,Clean,Topic
0,Isolating Complex Polynomial Roots: An algebra...,isolating complex polynomial roots an algebrai...,NaN
1,An algebraic algorithm to Isolate Complex poly...,an algebraic algorithm to isolate complex poly...,NaN
2,Using Computer Algebra and Chebyshev Polynomia...,using computer algebra and chebyshev polynomia...,NaN
3,Reviewing the SacarWeb design based in discoun...,reviewing the sacarweb design based discount u...,NaN
4,Extended Immersive Learning Environment: A Hyb...,extended immersive learning environment hybrid...,NaN
...,...,...,...
57973,A multicriteria approach for analysis of confl...,multicriteria approach for analysis of conflic...,NaN
57974,A Multicriteria Approach Using MAUT to Assist ...,multicriteria approach using maut to assist th...,NaN
57975,A Voting Approach Applied to Preventive Mainte...,voting approach applied to preventive maintena...,NaN
57976,Preference modeling experiments with surrogate...,preference modeling experiments with surrogate...,NaN


## Read Topics

In [27]:
arq = open(TOPIC_FILE, 'r')
topics = arq.readlines()
arq.close()

In [28]:
topics = topics[2:][:20]
topics

['approach method tool procedure technique algorithm framework strategy protocol scheme \n',
 'sqrts hadrons gev leptoquarks nbsptev ubar mesons zgamma pion mathbf \n',
 'brazil spain portugal argentina northeast south bahia parana amazon venezuela \n',
 'protog activityarrest prada prepchemsrc typesafe multiprotease hetererogeneity commitmentbased congestionaware applicabily \n',
 'analysis study evaluation investigation assessment prediction mapping modeling estimation aspects \n',
 'eigenmann imparfinis teleostei loricariidae ostariophysi callichthyidae leporinus anostomidae actinopterygii pisces \n',
 'software multiagent ontology hypermedia intelligent objectoriented componentbased metadata workflow ecommerce \n',
 'polytopes holomorphic euclidean embeddings graphs matroids ktheory sums hypergraphs automorphisms \n',
 'algorithm heuristic multiobjective problem pathrelinking grasp fuzzy iterative preconditioners equalizers \n',
 'learning education social teacher language thinking

In [45]:
df['Clean'].values[0].split(' ')

['isolating',
 'complex',
 'polynomial',
 'roots',
 'an',
 'algebraic',
 'algorithm',
 'using',
 'sturm',
 'sequences']

In [70]:
topics

['approach method tool procedure technique algorithm framework strategy protocol scheme \n',
 'sqrts hadrons gev leptoquarks nbsptev ubar mesons zgamma pion mathbf \n',
 'brazil spain portugal argentina northeast south bahia parana amazon venezuela \n',
 'protog activityarrest prada prepchemsrc typesafe multiprotease hetererogeneity commitmentbased congestionaware applicabily \n',
 'analysis study evaluation investigation assessment prediction mapping modeling estimation aspects \n',
 'eigenmann imparfinis teleostei loricariidae ostariophysi callichthyidae leporinus anostomidae actinopterygii pisces \n',
 'software multiagent ontology hypermedia intelligent objectoriented componentbased metadata workflow ecommerce \n',
 'polytopes holomorphic euclidean embeddings graphs matroids ktheory sums hypergraphs automorphisms \n',
 'algorithm heuristic multiobjective problem pathrelinking grasp fuzzy iterative preconditioners equalizers \n',
 'learning education social teacher language thinking

## Get Best Topics to Documents

In [76]:
def get_best_topic(topics, document):
    entropy = []

    for topic in topics:
        x=0
        topic_value = topic.split(' ')[:-1]
        for word in topic_value:
            if word in document:
                x += 1
        if x == 0:
            entropy.append(0)
        else:
            x_in = (len(topic)-x) / len(topic)
            x = x / len(topic)
            entropy.append((-x * math.log2(x))-(x_in * math.log2(x_in)))
    index_topic = entropy.index(max(entropy))
    select_topic = topics[index_topic]
    return select_topic

In [77]:
doc = df['Clean'].values[0].split(' ')
for index, _ in df.iterrows():
    df.loc[index, 'Topic'] = get_best_topic(topics, df.loc[index,'Clean'])
df

,Titles,Clean,Topic
0,Isolating Complex Polynomial Roots: An algebra...,isolating complex polynomial roots an algebrai...,approach method tool procedure technique algor...
1,An algebraic algorithm to Isolate Complex poly...,an algebraic algorithm to isolate complex poly...,approach method tool procedure technique algor...
2,Using Computer Algebra and Chebyshev Polynomia...,using computer algebra and chebyshev polynomia...,nonlinear finite approximation onedimensional ...
3,Reviewing the SacarWeb design based in discoun...,reviewing the sacarweb design based discount u...,modeling simulation design optimization theory...
4,Extended Immersive Learning Environment: A Hyb...,extended immersive learning environment hybrid...,patients children individuals women subjects a...
...,...,...,...
57973,A multicriteria approach for analysis of confl...,multicriteria approach for analysis of conflic...,approach method tool procedure technique algor...
57974,A Multicriteria Approach Using MAUT to Assist ...,multicriteria approach using maut to assist th...,approach method tool procedure technique algor...
57975,A Voting Approach Applied to Preventive Mainte...,voting approach applied to preventive maintena...,patients children individuals women subjects a...
57976,Preference modeling experiments with surrogate...,preference modeling experiments with surrogate...,approach method tool procedure technique algor...


In [78]:
df.to_csv('topic_link.csv',sep='|')